# Magnetic Grammar — Interactive Learner

An implementation of the feature-based phonological grammar from D'Alessandro & Van Oostendorp (2020).

**Core idea:** A language's phonology is characterised by features that *attract* or *reject* other features within a segment. The learner observes IPA segments and builds up these attract/reject properties, predicting which segments are well-formed.

## How to use this notebook

1. **Learn** — Feed the grammar IPA words or segments. The system learns features and their properties.
2. **Inspect** — View the current grammar (which features attract/reject which) and the predicted segment inventory.
3. **Check** — Test whether words are grammatical given the learned grammar.
4. **Reset** — Start over with a fresh grammar.

In [1]:
import sys, os
# Make sure the module is importable from this directory
sys.path.insert(0, os.path.dirname(os.path.abspath('magnetic_grammar_v2.py')))
sys.path.insert(0, '.')

from magnetic_grammar_v2 import MagneticGrammar, IPAInterface, FEATURE_NAMES

# Initialise the grammar
mg = MagneticGrammar()
print('Magnetic Grammar initialised.')
print(f'Using {len(mg.ipa.features)} curated privative features from panphon.')
print(f'Features: {sorted(FEATURE_NAMES[f] for f in mg.ipa.features)}')

Magnetic Grammar initialised.
Using 20 curated privative features from panphon.
Features: ['Anterior', 'Back', 'Consonantal', 'ConstrGlottis', 'Continuant', 'Coronal', 'DelayedRelease', 'Distributed', 'High', 'Labial', 'Lateral', 'Low', 'Nasal', 'Round', 'Sonorant', 'SpreadGlottis', 'Strident', 'Syllabic', 'Tense', 'Voice']


## Helper functions for display

These functions format the grammar, inventory, and learning traces as readable tables.

In [2]:
def show_grammar():
    """Display the current grammar as a formatted table."""
    rows = mg.grammar_table()
    if not rows:
        print('Grammar is empty. Learn some segments first!')
        return
    
    # Find max widths
    feat_w = max(len(r['feature']) for r in rows)
    attr_w = max(len(', '.join(r['attracts'])) for r in rows) if rows else 0
    rej_w = max(len(', '.join(r['rejects'])) for r in rows) if rows else 0
    
    header = f"{'Feature':<{feat_w}}  {'Attracts':<{max(attr_w,8)}}  {'Rejects':<{max(rej_w,7)}}"
    print(header)
    print('─' * len(header))
    for r in rows:
        attr_str = ', '.join(r['attracts']) if r['attracts'] else '—'
        rej_str = ', '.join(r['rejects']) if r['rejects'] else '—'
        print(f"{r['feature']:<{feat_w}}  {attr_str:<{max(attr_w,8)}}  {rej_str:<{max(rej_w,7)}}")

def show_inventory(basic_only=True):
    """Display the predicted segment inventory."""
    inv = mg.predicted_inventory(include_empty=True, basic_only=basic_only)
    if len(inv) <= 1:  # only empty segment
        print('No segments in inventory yet. Learn some segments first!')
        return
    
    print(f'Predicted inventory ({len(inv)} segments including ∅):\n')
    for item in inv:
        if item['is_empty']:
            print(f'  ∅  (empty segment — always valid, no phonetic realisation)')
        else:
            feat_str = ', '.join(item['feature_names'])
            print(f'  {item["ipa"]:>3}  {{{feat_str}}}')

def show_trace(trace):
    """Display a learning trace for one segment."""
    if trace is None:
        return
    seg = trace['segment']
    feat_names = sorted(mg.ipa.feature_name(f) for f in trace['features'])
    new_names = sorted(mg.ipa.feature_name(f) for f in trace['new_features'])
    
    feat_joined = ', '.join(feat_names)
    new_joined = ', '.join(new_names)
    
    print(f'\n── Learning /{seg}/ ──')
    print(f'   Features: {{{feat_joined}}}')
    
    if new_names:
        print(f'   New features introduced: {{{new_joined}}}')
        for f_code, n_props in trace['initial_properties'].items():
            fname = mg.ipa.feature_name(f_code)
            if n_props > 0:
                print(f'     {fname}: attracts {n_props} co-occurring known feature(s)')
            else:
                print(f'     {fname}: no prior features to constrain against')
    else:
        print('   No new features (all already known)')
    
    if trace['pruned']:
        print('   Pruned properties:')
        for f_code, props in trace['pruned'].items():
            for p in props:
                fname = mg.ipa.feature_name(f_code)
                print(f'     {fname}: removed {p}')
    else:
        print('   No properties pruned')

def show_check(ipa_word):
    """Display grammaticality check results for a word."""
    result = mg.check_word(ipa_word)
    verdict = '✓ grammatical' if result['valid'] else '✗ ungrammatical'
    print(f'  {ipa_word}  →  {verdict}')
    for d in result['details']:
        if not d['valid']:
            reasons = d.get('violations', [d.get('reason', '')])
            reasons_str = '; '.join(reasons)
            print(f'    ✗ /{d["segment"]}/: {reasons_str}')

print('Display functions ready.')

Display functions ready.


---
## 1. Learning segments

Feed the grammar IPA words. Each unique segment is learned in order.
The system shows step-by-step what happens internally:
- Which features are new
- What attract/reject properties are initially posited
- Which properties get pruned by the observed segment

### Example: learning a simple consonant system

Let's start by teaching the grammar the consonants /s/ and /p/, mimicking the paper's example.

In [3]:
# Learn /s/ - our first segment
trace = mg.learn_segment('s')
show_trace(trace)

print('\n── Grammar after /s/ ──')
show_grammar()

print('\n── Inventory after /s/ ──')
show_inventory()


── Learning /s/ ──
   Features: {Anterior, Consonantal, Continuant, Coronal, Strident}
   New features introduced: {Anterior, Consonantal, Continuant, Coronal, Strident}
     Consonantal: no prior features to constrain against
     Continuant: no prior features to constrain against
     Strident: no prior features to constrain against
     Anterior: no prior features to constrain against
     Coronal: no prior features to constrain against
   No properties pruned

── Grammar after /s/ ──
Feature      Attracts  Rejects
──────────────────────────────
Anterior     —         —      
Consonantal  —         —      
Continuant   —         —      
Coronal      —         —      
Strident     —         —      

── Inventory after /s/ ──
Predicted inventory (5 segments including ∅):

  ∅  (empty segment — always valid, no phonetic realisation)
    ʈ  {Consonantal, Coronal}
    t  {Anterior, Consonantal, Coronal}
    ʂ  {Consonantal, Continuant, Coronal, Strident}
    s  {Anterior, Consonantal, C

In [4]:
# Learn /p/ - introduces Labial
trace = mg.learn_segment('p')
show_trace(trace)

print('\n── Grammar after /s, p/ ──')
show_grammar()

print('\n── Inventory after /s, p/ ──')
show_inventory()


── Learning /p/ ──
   Features: {Anterior, Consonantal, Labial}
   New features introduced: {Labial}
     Labial: attracts 2 co-occurring known feature(s)
   No properties pruned

── Grammar after /s, p/ ──
Feature      Attracts               Rejects
───────────────────────────────────────────
Anterior     —                      —      
Consonantal  —                      —      
Continuant   —                      —      
Coronal      —                      —      
Labial       Anterior, Consonantal  —      
Strident     —                      —      

── Inventory after /s, p/ ──
Predicted inventory (8 segments including ∅):

  ∅  (empty segment — always valid, no phonetic realisation)
    ʈ  {Consonantal, Coronal}
    t  {Anterior, Consonantal, Coronal}
    p  {Anterior, Consonantal, Labial}
    ʂ  {Consonantal, Continuant, Coronal, Strident}
    ɸ  {Anterior, Consonantal, Continuant, Labial}
    s  {Anterior, Consonantal, Continuant, Coronal, Strident}
    f  {Anterior, Consonanta

Notice how the grammar has already generalised: after learning just /s/ and /p/, it predicts /t/ and /f/ as well. Features from the first segment are unconstrained relative to each other (they entered together, so there's no evidence they *need* each other). The only constraints come from Labial, which entered later and attracts the features it co-occurred with (Anterior, Consonantal).

To see all valid feature bundles including diacriticked IPA symbols, call `show_inventory(basic_only=False)`.

In [5]:
# Learn a vowel
trace = mg.learn_segment('a')
show_trace(trace)

print('\n── Inventory after /s, p, a/ ──')
show_inventory()


── Learning /a/ ──
   Features: {Back, Continuant, Low, Sonorant, Syllabic, Tense, Voice}
   New features introduced: {Back, Low, Sonorant, Syllabic, Tense, Voice}
     Back: attracts 1 co-occurring known feature(s)
     Voice: attracts 1 co-occurring known feature(s)
     Syllabic: attracts 1 co-occurring known feature(s)
     Sonorant: attracts 1 co-occurring known feature(s)
     Low: attracts 1 co-occurring known feature(s)
     Tense: attracts 1 co-occurring known feature(s)
   No properties pruned

── Inventory after /s, p, a/ ──
Predicted inventory (13 segments including ∅):

  ∅  (empty segment — always valid, no phonetic realisation)
    ʈ  {Consonantal, Coronal}
    t  {Anterior, Consonantal, Coronal}
    p  {Anterior, Consonantal, Labial}
    ʂ  {Consonantal, Continuant, Coronal, Strident}
    ɻ  {Continuant, Coronal, Sonorant, Voice}
    ɛ  {Continuant, Sonorant, Syllabic, Voice}
    s  {Anterior, Consonantal, Continuant, Coronal, Strident}
    ə  {Back, Continuant, Sonora

### Expanding the system

Following the paper, let's add /z/, /b/, and /f/ to see the inventory grow.

In [6]:
for seg in ['z', 'b', 'f']:
    trace = mg.learn_segment(seg)
    show_trace(trace)

print('\n── Grammar now ──')
show_grammar()

print('\n── Full inventory ──')
show_inventory()


── Learning /z/ ──
   Features: {Anterior, Consonantal, Continuant, Coronal, Strident, Voice}
   No new features (all already known)
   Pruned properties:
     Anterior: removed reject(Voice)
     Consonantal: removed reject(Voice)

── Learning /b/ ──
   Features: {Anterior, Consonantal, Labial, Voice}
   No new features (all already known)
   Pruned properties:
     Voice: removed attract(Continuant)
     Voice: removed reject(Labial)

── Learning /f/ ──
   Features: {Anterior, Consonantal, Continuant, Labial, Strident}
   No new features (all already known)
   Pruned properties:
     Strident: removed reject(Labial)
     Labial: removed reject(Continuant)
     Labial: removed reject(Strident)
     Continuant: removed reject(Labial)

── Grammar now ──
Feature      Attracts               Rejects                                      
─────────────────────────────────────────────────────────────────────────────────
Anterior     —                      Back, Low, Sonorant, Syllabic, Tense

As the paper predicts, /v/ emerges automatically: the language has a voiced fricative (/z/), a labial fricative (/f/), and a voiced labial (/b/), so /v/ must also be valid — there's no single feature that can reject it.

---

## 2. Learning from words

You can also feed whole IPA words. The system will segment them and learn each segment.

In [7]:
# Reset and learn from words instead
mg.reset()
print('Grammar reset.\n')

# Learn from Dutch-like words
words = ['pat', 'zat', 'bad', 'fas']
for word in words:
    print(f'═══ Learning word: {word} ═══')
    traces = mg.learn_word(word)
    for t in traces:
        show_trace(t)
    print()

print('\n── Grammar ──')
show_grammar()
print('\n── Predicted inventory ──')
show_inventory()

Grammar reset.

═══ Learning word: pat ═══

── Learning /p/ ──
   Features: {Anterior, Consonantal, Labial}
   New features introduced: {Anterior, Consonantal, Labial}
     Consonantal: no prior features to constrain against
     Anterior: no prior features to constrain against
     Labial: no prior features to constrain against
   No properties pruned

── Learning /a/ ──
   Features: {Back, Continuant, Low, Sonorant, Syllabic, Tense, Voice}
   New features introduced: {Back, Continuant, Low, Sonorant, Syllabic, Tense, Voice}
     Back: no prior features to constrain against
     Voice: no prior features to constrain against
     Continuant: no prior features to constrain against
     Syllabic: no prior features to constrain against
     Sonorant: no prior features to constrain against
     Low: no prior features to constrain against
     Tense: no prior features to constrain against
   No properties pruned

── Learning /t/ ──
   Features: {Anterior, Consonantal, Coronal}
   New featur

---

## 3. Checking grammaticality

Check whether words consist only of valid segments.

In [8]:
# Check various words
print('Grammaticality checks:\n')
test_words = ['pat', 'sat', 'bad', 'kap', 'map', 'faz', 'vis']
for w in test_words:
    show_check(w)

Grammaticality checks:

  pat  →  ✓ grammatical
  sat  →  ✓ grammatical
  bad  →  ✓ grammatical
  kap  →  ✗ ungrammatical
    ✗ /k/: Consonantal: reject(Back) violated; Feature 'High' not in grammar
  map  →  ✗ ungrammatical
    ✗ /m/: Feature 'Nasal' not in grammar; Labial: reject(Sonorant) violated; Anterior: reject(Sonorant) violated; Consonantal: reject(Sonorant) violated
  faz  →  ✓ grammatical
  vis  →  ✗ ungrammatical
    ✗ /i/: Feature 'High' not in grammar


---

## 4. Your own experiments

Use the cells below to run your own experiments. You can:

- `mg.reset()` — start fresh
- `mg.learn_word('...')` — learn from an IPA word  
- `mg.learn_segment('...')` — learn a single IPA segment
- `show_grammar()` — display the current grammar
- `show_inventory()` — display the predicted segment inventory
- `show_check('...')` — check if a word is grammatical
- `show_trace(mg.learn_segment('...'))` — learn and see what changed

In [9]:
# YOUR EXPERIMENT: reset and try your own language
mg.reset()

# Example: learn some segments of Hawaiian
# Hawaiian has: p, k, ʔ, h, m, n, l, w, a, e, i, o, u
hawaiian_words = ['aloha', 'wiki', 'poke', 'mana', 'hula', 'lani']

for word in hawaiian_words:
    traces = mg.learn_word(word)
    inv = mg.predicted_inventory()
    n_segs = len([x for x in inv if not x['is_empty']])
    print(f'After "{word}": {n_segs} segments in inventory')

print('\n── Grammar ──')
show_grammar()
print('\n── Predicted inventory ──')
show_inventory()

After "aloha": 15 segments in inventory
After "wiki": 32 segments in inventory
After "poke": 39 segments in inventory
After "mana": 41 segments in inventory
After "hula": 42 segments in inventory
After "lani": 42 segments in inventory

── Grammar ──
Feature      Attracts                                Rejects                                                     
─────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Anterior     —                                       Back, High, Low, Round, Syllabic, Tense                     
Back         —                                       Anterior, Coronal, Lateral, Nasal                           
Consonantal  —                                       Low, Round, Syllabic, Tense                                 
Continuant   —                                       Nasal                                                       
Coronal      Sonorant, Voice                         Back, High, L

In [10]:
# Check grammaticality of Hawaiian words
print('Hawaiian word checks:\n')
for w in ['aloha', 'wiki', 'poke', 'strik', 'blam']:
    show_check(w)

Hawaiian word checks:

  aloha  →  ✓ grammatical
  wiki  →  ✓ grammatical
  poke  →  ✓ grammatical
  strik  →  ✗ ungrammatical
    ✗ /s/: Feature 'Strident' not in grammar; Coronal: attract(Sonorant) violated; Coronal: attract(Voice) violated
    ✗ /t/: Coronal: attract(Sonorant) violated; Coronal: attract(Voice) violated
  blam  →  ✓ grammatical


In [11]:
# TRY YOUR OWN LANGUAGE HERE
mg.reset()

# Input your IPA words below:
my_words = ['...']  # Replace with your IPA words

for word in my_words:
    traces = mg.learn_word(word)
    for t in traces:
        show_trace(t)

print('\n── Grammar ──')
show_grammar()
print('\n── Predicted inventory ──')
show_inventory()


── Grammar ──
Grammar is empty. Learn some segments first!

── Predicted inventory ──
No segments in inventory yet. Learn some segments first!


In [12]:
# Check your words here:
my_test_words = ['...']  # Replace with IPA words to check

for w in my_test_words:
    show_check(w)

  ...  →  ✓ grammatical


---

## Quick reference: IPA input

You can type most IPA symbols directly. Some useful ones:

| Symbol | Description | Symbol | Description |
|--------|-------------|--------|-------------|
| ʃ | voiceless postalveolar fricative | ʒ | voiced postalveolar fricative |
| θ | voiceless dental fricative | ð | voiced dental fricative |
| ŋ | velar nasal | ɡ | voiced velar stop |
| ʔ | glottal stop | ɾ | alveolar tap |
| ɛ | open-mid front vowel | ɔ | open-mid back rounded vowel |
| ə | schwa | æ | near-open front vowel |
| ɪ | near-close front vowel | ʊ | near-close back vowel |

The system uses **panphon** for IPA segmentation, so most standard IPA (including diacritics) should work.